# Scraper


In [1]:
# Construct my start_urls for all the cases.
# As there are no "previous/next" link betweem cases,
# we cannot use a typical recursive algorithm.
# Thus, we use the fact that the case names are structured:
# year + number + year
# (This is surely not the best way!)

last_year_in_data = 2018 # to be updated

years = list(range(2018, last_year_in_data+1))
years = [str(y) for y in years] # convert to string to concatenate with case numbers

case_num = list(range(1, 60))
# convert to string to concatenate with years:
case_num = ['-' + str(c).zfill(3) + 'N' for c in case_num] # zfill adds 0 to the left

# concatenate all string and years together:
cases = []
for y in years:
    for c in case_num:
        str(cases.append(y + c))

# complete URLs:
cases = ['https://www.ekr.admin.ch/f524/' + case + '.html' for case in cases]


### Issues to check:

* 2017-029N is missing
    * because no history? 

* 2016-020N missing too

* have duplicates: 2015-043N, 2015-047N, 2017-010N (4), 2018-009N, 2018-015N (3), 

* the accents are not read well by Excel. Excel issue?

* ajouter les critères de recherche juridiques

__Decisions:__  


In [2]:
# Importing in each cell because need to restart kernel
import scrapy # version 2.2.0
import time
from scrapy.crawler import CrawlerProcess

t0 = time.time()
print("Running...")

class CFRSpider(scrapy.Spider):
    name = "CFR" # Naming the spider if you  running more than one spider of this class simultaneously.
    allowed_domains = ['https://www.ekr.admin.ch/f524/']
#     start_urls = [
#         'https://www.ekr.admin.ch/f524/2014-005N.html',
#         'https://www.ekr.admin.ch/f524/2016-020N.html'
#     ]
    start_urls = cases
    
    def parse(self, response):
        
        item = {} # a dictionary to store the results
        
        
        item['case'] = response.xpath('body/div/div[2]/div/div/div/div/div/p/text()').extract_first()
        item['name'] = response.xpath('string(body/div/div[2]/div/div/div/div/div/h2)').extract_first() # string(.) instead of ./text() is useful to get the <i>
        item['location'] = response.xpath('body/div/div[2]/div/div/div/div/div/p[2]/text()').extract_first()
        item['url'] = response.request.url

        
        # procedure history | Historique de la procédure | Verfahrensgeschichte | Cronistoria della procedura
        tr_list = []
        # we list all related cases in history (year, name, decision), separated by a |
        for cnt, tr_selector in enumerate(response.xpath('body/div/div[2]/div/div/div/div/div/table[1]/tr'), start=1):
            tr_list.append(tr_selector.xpath('normalize-space()').get())
        item['history'] = '|'.join(tr_list)
        
        
        
        # TODO: add here the critères de recherche juridiques
        
        
        
        # Keywords | Mots-clés | Stichwörter | Parole chiave
        # there are at maximum these 5:
        item['authors'] = response.xpath('string(body/div/div[2]/div/div/div/div/div/table[3]/tr[1]/td[2])').extract_first()
        item['victims'] = response.xpath('string(body/div/div[2]/div/div/div/div/div/table[3]/tr[2]/td[2])').extract_first()
        item['means'] = response.xpath('string(body/div/div[2]/div/div/div/div/div/table[3]/tr[3]/td[2])').extract_first()
        item['social_env'] = response.xpath('string(body/div/div[2]/div/div/div/div/div/table[3]/tr[4]/td[2])').extract_first()
        item['ideology'] = response.xpath('string(body/div/div[2]/div/div/div/div/div/table[3]/tr[5]/td[2])').extract_first()

        
        # The Text describing the case
        text = [] # collect all the text with html tags in one list
        # we do this, because there is no regular structure in the texts displayed
        
        # h3 titles that are not part of the interesting data:
        titles_not_wanted = ['<h3>Actualité</h3>', '<h3>Thèmes</h3>', '<h3>Bases juridiques</h3>', '<h3>International</h3>', '<h3>Prestations</h3>', '<h3>Publications</h3>', '<h3>La CFR</h3>', '<h3>Restez informé</h3>']
        
        # loop through all h3 titles and get them, as well as all the <p> in between:
        for cnt, h3_selector in enumerate(response.css('h3'), start=1):
            key = h3_selector.extract() # <-with html tags | without-> xpath('normalize-space()').get() # get the h3 title's text
            if key not in titles_not_wanted:
                text.append(key) # append the h3 title
                values = h3_selector.xpath('following-sibling::p[count(preceding-sibling::h3)=$cnt]', cnt=cnt).getall()
                values = ''.join([str(elem) for elem in values])  # convert list to string (remove [])
                text.append(values) # add the paragraphs <p> in between the <h3>
        
        item['html_text'] = ''.join(text) # all the text together, in a single string (with html tags)
        item['html_text_as_list'] = text # all the text together, as a list (with html tags)
        
        
        yield(item)
    
    
# Instantiate our crawler.
process = CrawlerProcess({
    'FEED_FORMAT': 'json',         # Store data in JSON format.
    'FEED_URI': 'test.json',       # Name our storage file.
    'LOG_ENABLED': False           # Turn off logging for now.
})

# Start the crawler with our spider.
process.crawl(CFRSpider)
process.start()
print("Done! It took " + str(time.time() - t0) + " seconds." )

Running...
Done! It took 10.005990028381348 seconds.


In [3]:
import pandas as pd

test = pd.read_json('test.json', orient='records')
print(test.shape)
test.head().T

(59, 12)


,0,1,2,3,4
case,Cas 2018-003N,Cas 2018-004N,Cas 2018-008N,Cas 2018-002N,Cas 2018-010N
name,« Mir bedienet kei Neger »,Commentaire ironique : bombes dans les mosquées,Provokativen Postkarten von Sharia_pride II,Internationale Konferenz der Anti-Zensur-Koali...,Rassistisches Video
location,St-Gall,Vaud,Schwyz,Grisons,Berne
url,https://www.ekr.admin.ch/f524/2018-003N.html,https://www.ekr.admin.ch/f524/2018-004N.html,https://www.ekr.admin.ch/f524/2018-008N.html,https://www.ekr.admin.ch/f524/2018-002N.html,https://www.ekr.admin.ch/f524/2018-010N.html
history,2018 2018-003N Die zuständige Strafverfolgungs...,2018 2018-004N L’autorité compétente en matièr...,2018 2018-008N Die zuständige Strafverfolgungs...,2018 2018-002N Der Beschuldigte wird der Rasse...,2018 2018-010N Die zuständige Strafverfolgungs...
authors,Acteurs du secteur tertiaire,Particuliers,Particuliers;\nExtrémistes de droite,Acteurs politiques;\nParticuliers;\nExtrémiste...,Particuliers
victims,Noirs / personnes de couleur,Musulmans,Musulmans,Juifs,Noirs / personnes de couleur
means,Déclarations orales;\nRefus de prestations,Ecrits;\nCommunication électronique,Ecrits;\nSons / images;\nPropagation de matéri...,Déclarations orales;\nEcrits;\nSons / images;\...,Déclarations orales;\nEcrits;\nCommunication é...
social_env,Lieux publics,Mass media (Internet inclus);\nMédias sociaux,Voisinage,Associations / Fédérations / Organisations,Mass media (Internet inclus)
ideology,Racisme (couleur de peau),Islamophobie,Islamophobie,Antisémitisme;\nRévisionnisme,Racisme (couleur de peau)


In [3]:
# test["En droit / considérants_4"][0]
test.head()
list(test.history)


['2014 2014-005N Die 1. Instanz verurteilt den Angeklagten.|2015 2015-048N Die 2. Instanz verurteilt den Beschuldigten.']

In [6]:
test.html_text_as_list[12]

['<h3>Synthèse</h3>',
 '<p>Der Beschuldigte hat auf einem Fussballplatz in Anwesenheit von Kindern und anderen Zuschauern einen Mann als « <i>Dreckneger</i> » betitelt und ihm gesagt « <i>Wenn ich eine Banane werfen würde, würdest du wie ein Affe die Banane holen</i> ».Diese öffentliche Herabsetzung in der Menschenwürde verstösst nach Meinung der zuständigen Strafverfolgungsbehörde gegen das Verbot der Rassendiskriminierung in Art. 261<sup>bis</sup> Abs. 4 StGB, weshalb der Beschuldigte zu einer Geldstrafe verurteilt wurde.<br>\n</p>',
 '<h3>Décision</h3>',
 '<p>Der Beschuldigte wird wegen Rassendiskriminierung (Art. 261<sup>bis</sup> Abs. 4 StGB), für schuldig erklärt und wird mit einer Geldstrafe von 30 Tagessätzen zu je CHF 50.00, ausmachend CHF 300.00 bestraft. Die Geldstrafe wird bedingt ausgesprochen bei einer Probezeit von 3 Jahren. Die Kosten des Verfahrens im Umfang von CHF 530.00 werden dem Beschuldigten auferlegt.<br>\n</p>']

In [ ]:
# some cleaning 

In [6]:
# save data table as .csv

test.to_csv(path_or_buf='test.csv')